In [1]:
import dremio_client.lib as dlib
import numpy as np

import pandas as pd
import h3pandas
from datetime import datetime
import time

from pyarrow import flight
from pyarrow.flight import FlightClient
import pyarrow.dataset as ds
import polars as pl

from rapidfuzz import fuzz
import datetime
from dask.distributed import Client, LocalCluster
import dask.dataframe as dd


In [2]:
def get_bakong_data():

    query = f"""
        SELECT 
        
            DISTINCT MerchantName
            
        FROM "DataScience_DB"."General_DB".dbo.bk_outgoing
        where date_trunc('month',created_at) = '2023-12-01'
        and type = 'QR' AND MerchantType = '29'
        AND src_name <> MerchantName
        
        ORDER BY 1
        
    """

    data = dlib.simple_query(query)

    return data

In [3]:
def get_aba_data():
    query = f"""
        SELECT DISTINCT CIF, AC_NAME
        FROM dwh.FCCBOREP.VW_CUSTACC
        ORDER BY 1
    """
    data = dlib.simple_query(query)
    return data



In [4]:
#bakong_data = get_bakong_data()

aba_data_set = get_aba_data()
aba_data_set.to_csv('dataset/aba_data.csv')

In [ ]:
aba_data = aba_data_set

In [ ]:
#bakong_data.shape
aba_data.shape

<hr>
Clean ABA Data
<hr>

In [ ]:
aba_data.tail()

In [ ]:
aba_data.info()

In [ ]:
aba_data['AC_NAME'] = aba_data['AC_NAME'].str.upper()
aba_data['AC_NAME'] = aba_data['AC_NAME'].str.strip()
aba_data['AC_NAME'] = aba_data['AC_NAME'].str.replace(" ", "")

In [ ]:
aba_data = aba_data.drop_duplicates()
aba_data = aba_data.dropna()

In [ ]:
aba_data.shape

<hr>
Clean Bakong Data
<hr>


In [ ]:
bakong_data = pd.read_csv('dataset/unique_merchantname_bk_500001_lastrow.csv')
bakong_data.shape

In [ ]:
bakong_data.drop('Unnamed: 0', axis=1, inplace=True)
bakong_data.head()

In [ ]:
bakong_data['Unique_MerchantName'] = bakong_data['Unique_MerchantName'].str.upper()
bakong_data['Unique_MerchantName'] = bakong_data['Unique_MerchantName'].str.strip()
bakong_data['Unique_MerchantName'] = bakong_data['Unique_MerchantName'].str.replace(" ", "")

bakong_data.shape

In [ ]:
bk_cust = bakong_data.head(1000)
aba_cust = aba_data.head(10000)

In [ ]:
print(bk_cust.shape)
print(aba_cust.shape)

In [ ]:
#concate the dataframe 
data = pd.DataFrame({"bakong_cust" : bk_cust['Unique_MerchantName'] , "aba_cust" : aba_cust['AC_NAME']})
data.head()

In [ ]:
data.dropna(subset=['aba_cust'], inplace=True)
#data.dropna(subset=['bakong_cust'], inplace=True)

data.head()

In [ ]:
import re
from langdetect import detect, LangDetectException

def clean_text(text):
    
    if not isinstance(text, str):
        text = ''
    cleaned_text = re.sub(r'[^a-zA-Z.\- ]+', '', text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text
    
def is_english(text):
    try:
        return detect(text) == 'en'
    except LangDetectException:
        return False

In [ ]:

data['bakong_cust'] = data['bakong_cust'].apply(clean_text)
data['aba_cust'] = data['aba_cust'].apply(clean_text)


In [ ]:
data.head()

<hr>
Dask Python - Data Frame 
<hr>

In [ ]:
data.shape

In [ ]:

from rapidfuzz import process, fuzz
import pandas as pd

def apply_rapidfuzz_matching(df, aba_cust_list):
    df['bakong_cust'] = df['bakong_cust'].astype(str)

    def apply_fuzzy(x):
        match = process.extractOne(x, aba_cust_list, scorer=fuzz.token_set_ratio, score_cutoff=95)
        return match if match else (None, None, None)  
    
    df['match_results'] = df['bakong_cust'].apply(apply_fuzzy)
    
    results = pd.DataFrame(df['match_results'].tolist(), index=df.index)
    
    results.columns = ['aba_cust', 'score', 'other']

    final_results = df[['bakong_cust']].join(results[['aba_cust', 'score']])
    
    # Drop rows without a match
    return final_results.dropna(subset=['aba_cust', 'score'])


In [ ]:
# Convert the Pandas dataframe to a Dask dataframe
dask_df = dd.from_pandas(data, npartitions=500)
dask_df

In [ ]:
client = Client(n_workers=10, memory_limit="5GB")
client

In [ ]:
#create dask data to list
aba_cust_list = dask_df['aba_cust'].compute().tolist()

# save the dask data above to each cluster to process bakong data
#distributed_aba_cust_list = client.scatter(aba_cust_list, broadcast=True)
#create meta 
meta = {'bakong_cust': 'object', 'aba_cust': 'object', 'score': 'float'}

In [ ]:

# apply the data and do matching 
result_2 = dask_df.map_partitions(apply_rapidfuzz_matching, aba_cust_list, meta=meta)

start = datetime.datetime.now()
print("Start time:", datetime.datetime.now())

results_df_2 = result_2.compute()

end = datetime.datetime.now()

print("End time:", end)
print("Duration:", end - start)


In [ ]:
results_df_2

<hr>
Terminal Cluster
<hr>